# What's in the notebook?
**1)EDA**

**2)Detection of missing data and imputation.**

**3)Feature enginnering.**

**4)Coverting categorical features.**

**5)Modelling.**

**6)Ensemble.**

**7)K-fold cv.**

**8)Hyperparameter tuning.**

# What's new?
**1. Removed executon logs to increase the readability.**

# Note:
**This is a work in progress.Therefore,your feedback is valuable for me.**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

In [ ]:
train= pd.read_csv("/kaggle/input/titanic/train.csv")
test =pd.read_csv("/kaggle/input/titanic/test.csv")
ss =pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [ ]:
train.head()

# Types of features in titanic

**Categorical**: is a collection of information that is divided into groups.
Eg. Embarked (C = Cherbourg; Q = Queenstown; S = Southampton)

**Ordinal**: They are similar to categorical features but they have an order.
Eg. Pclass (1, 2, 3)

**Binary**: A categrorical feature which has only 2 types of categories.Which is often represted as 0 and 1.
Eg: Sex (Male/Female)

**Continuous**: They can take up any value between the minimum and maximum values in a column.
Eg. Age, Fare

**Count**: They represent the count of a variable.
Eg. SibSp, Parch

# Please upvote if you like this notebook.

In [ ]:
test.head()

In [ ]:
train.describe()

Above,we can see that the age of passengers ranges between 0.4 to 80 years.And 38% of the people survived 
whose records are mentioned in this training set.

<h1><center>Titanic EDA</center></h1>

In [ ]:
sns.catplot(x='Pclass', data=train,kind='count', hue='Sex',palette='rocket')

Class 3 males are more than 2x of women.

In [ ]:
sns.catplot(x = 'Sex',data=train, kind='count',palette='rocket')

The number of male passengers are almost 2x of female passengers appox.

In [ ]:
sns.barplot(x='Pclass', y='Survived', data=train,palette='rocket')

Here,we can see that person in class 1 has the higher chances of survival.Where as person in class 3 has lower chance.

In [ ]:
sns.catplot(x ='Survived',data=train, kind='count',hue='Sex',palette='rocket')

Despite the ship being male dominated.The survival rate of women is higher.As we can see that large number of women survived despite being minorities.

In [ ]:
sns.catplot(x='Pclass', data=train,kind='count', hue='Sex',palette='rocket')

In class 3 males are more than 2x of women.

In [ ]:
#Age
sns.boxplot(x='Survived',y='Age',data=train,palette='winter')

The average age of people seems to almost same for survived and dead.however we can observe some outliers

In [ ]:
sns.displot(train['Age'].dropna(),kde=False,color='darkred',bins=40)

Age distrubution in titanic traning dataset

In [ ]:
#Survival and dead distribution with respect to age
plt.figure(figsize=(20, 30))
sns.countplot(y = "Age",hue="Survived", data=train)

In [ ]:
FacetGrid = sns.FacetGrid(train, row='Embarked', size=5, aspect=1.6)
FacetGrid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette=None,  order=None, hue_order=None )
FacetGrid.add_legend()

Women on port Q and S has higher probability of surviving,Same cannot be said for the women on port C.The probability of men surviving is lower on port Q and S and it is higher in C.

In [ ]:
print(train["SibSp"].value_counts())

In [ ]:
ax = sns.countplot(x = "SibSp",hue="Survived", data=train,palette='rocket')

From the above plot we can see that people with 0 and 1 Sibsp(Sibling or spouse) has greater chance of survival.Maybe because of sibsp with 0 or 1 were present in greater number.

In [ ]:
sns.heatmap(train.corr(method="spearman"));

Spearman corr of training data

<h1><center>Handling Missing values and imputing it</center></h1>

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap="viridis")

All the yellow lines that we see are missing values.We can see that majority of values are missing in cabin and age.

In [ ]:
total = train.isnull().sum().sort_values(ascending=False)
total

These are the following missing values.

In [ ]:
train.head()

In [ ]:
train.drop(['Name','Ticket','Cabin','Fare'],axis=1,inplace=True)
test.drop(['Name','Ticket','Cabin','Fare'],axis=1,inplace=True)

In [ ]:
train

**Relation ship between PCLASS and AGE**

In [ ]:
sns.boxplot(x='Pclass',y='Age',data=train)

Filling up mean values for null.

In [ ]:
sns.boxplot(x='Pclass',y='Age',data=test)

In [ ]:
def impute_age_train(cols):
    Age=cols[0]
    Pclass=cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else :
            return 24
    else:
        return Age

In [ ]:
def impute_age_test(cols):
    Age=cols[0]
    Pclass=cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 42
        elif Pclass == 2:
            return 27
        else :
            return 25
    else:
        return Age

In [ ]:
train['Age']=train[['Age','Pclass']].apply(impute_age_train,axis=1)
test['Age']=test[['Age','Pclass']].apply(impute_age_test,axis=1)

In [ ]:
train['Embarked'].describe()

S is common in Embarked.We will fill null values with it.

In [ ]:
common='S'
train['Embarked'] = train['Embarked'].fillna(common)

In [ ]:
train.isnull().sum()


# Catogorical features

In [ ]:
Embarked = pd.get_dummies(train['Embarked'],drop_first=True)
Embarked_test = pd.get_dummies(test['Embarked'],drop_first=True)
Embarked.head()

In [ ]:
#Converting values
train['Sex']=train['Sex'].map({'male':0,'female':1})
test['Sex']=test['Sex'].map({'male':0,'female':1})
train.drop(['Embarked'],axis=1,inplace=True)
test.drop(['Embarked'],axis=1,inplace=True)

In [ ]:
train

In [ ]:
test

In [ ]:
train = pd.concat([train,Embarked],axis=1)
test = pd.concat([test,Embarked_test],axis=1)
train.head()

In [ ]:
train.loc[train['Age'] <= 16, 'Age'] = 0
train.loc[(train['Age'] > 16) & (train['Age'] <= 32), 'Age'] = 1
train.loc[(train['Age'] > 32) & (train['Age'] <= 48), 'Age'] = 2
train.loc[(train['Age'] > 48) & (train['Age'] <= 64), 'Age'] = 3
train.loc[ train['Age'] > 64, 'Age'] = 4

In [ ]:
test.loc[test['Age'] <= 16, 'Age'] = 0
test.loc[(test['Age'] > 16) & (test['Age'] <= 32), 'Age'] = 1
test.loc[(test['Age'] > 32) & (test['Age'] <= 48), 'Age'] = 2
test.loc[(test['Age'] > 48) & (test['Age'] <= 64), 'Age'] = 3
test.loc[ test['Age'] > 64, 'Age'] = 4

In [ ]:
train= train.drop("PassengerId", axis = 1)

In [ ]:
test.head()

<h1><center>MODELLING</center></h1>

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split


In [ ]:
X_train = train.drop("Survived", axis=1)
Y_train = train["Survived"]
X_test = test.drop("PassengerId", axis = 1)


# Ensemble technique

In [ ]:
KNN=KNeighborsClassifier()
NAIVE=GaussianNB()
SVM=SVC()
DT=DecisionTreeClassifier()
LR = LogisticRegression()
RF = RandomForestClassifier()
Ensemble = VotingClassifier( estimators= [('KNN',KNN),('NB',NAIVE),('SVM',SVM),('DT',DT),('LR',LR),('RF',RF)], voting = 'hard')

In [ ]:
Ensemble.fit(X_train,Y_train)

In [ ]:
from sklearn import metrics
Y_pred_rand = (Ensemble.predict(X_train) > 0.5).astype(int)
print('Precision : ', np.round(metrics.precision_score(Y_train, Y_pred_rand)*100,2))
print('Accuracy : ', np.round(metrics.accuracy_score(Y_train, Y_pred_rand)*100,2))
print('Recall : ', np.round(metrics.recall_score(Y_train, Y_pred_rand)*100,2))
print('F1 score : ', np.round(metrics.f1_score(Y_train, Y_pred_rand)*100,2))
print('AUC : ', np.round(metrics.roc_auc_score(Y_train, Y_pred_rand)*100,2))

In [ ]:
catboost = CatBoostClassifier(silent=True)
catboost.fit(X_train, Y_train)
Y_pred = catboost.predict(X_test)
acc_catboost = round(catboost.score(X_train, Y_train) * 100, 2);

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [ ]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

In [ ]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [ ]:
random_forest = RandomForestClassifier(n_estimators = 100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

In [ ]:

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

In [ ]:
models = pd.DataFrame({'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
                                 'Random Forest', 'Naive Bayes', 'Perceptron', 'Stochastic Gradient Decent', 
                                 'Linear SVC', 'Decision Tree', 'CatBoost'],
                       'Score': [acc_svc, acc_knn, acc_log, acc_random_forest, acc_gaussian, acc_perceptron,
                                 acc_sgd, acc_linear_svc, acc_decision_tree, acc_catboost]})

models.sort_values(by = 'Score', ascending = False, ignore_index = True)

In [ ]:
classifiers = []
classifiers.append(LogisticRegression())
classifiers.append(SVC())
classifiers.append(KNeighborsClassifier(n_neighbors = 5))
classifiers.append(GaussianNB())
classifiers.append(Perceptron())
classifiers.append(LinearSVC())
classifiers.append(SGDClassifier())
classifiers.append(DecisionTreeClassifier())
classifiers.append(RandomForestClassifier())
classifiers.append(CatBoostClassifier(silent=True))

len(classifiers)

In [ ]:
from sklearn.model_selection import cross_val_score

# k-fold Cross-validation

In [ ]:
cv_results = []
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier, X_train, Y_train, scoring = 'accuracy',verbose=False))
    

In [ ]:
cv_mean = []
cv_std = []
for cv_result in cv_results:
    cv_mean.append(cv_result.mean())
    cv_std.append(cv_result.std())

In [ ]:
cv_res = pd.DataFrame({'Cross Validation Mean': cv_mean, 'Cross Validation Std': cv_std, 'Algorithm': ['Logistic Regression', 'Support Vector Machines', 'KNN', 'Gausian Naive Bayes', 'Perceptron', 'Linear SVC', 'Stochastic Gradient Descent', 'Decision Tree', 'Random Forest', 'CatBoost']})
cv_res.sort_values(by = 'Cross Validation Mean', ascending = False, ignore_index = True)

In my case i got support vector machine with more accuracy.

In [ ]:
sns.barplot('Cross Validation Mean', 'Algorithm', data = cv_res, order = cv_res.sort_values(by = 'Cross Validation Mean', ascending = False)['Algorithm'], palette = 'Set3', **{'xerr': cv_std})
plt.ylabel('Algorithm')
plt.title('Cross Validation Scores')

**Svc has higher cv score**

# Hyperparameter tuning

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 0) 

grid.fit(X_train, Y_train)

In [ ]:
print("Best parameters: ", grid.best_params_) 
print("Best estimator: ", grid.best_estimator_)

In [ ]:
svc = SVC(C = 100, gamma = 0.01, kernel = 'rbf')
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In [ ]:
cross_val_score(svc, X_train, Y_train, scoring = 'accuracy', cv = 10).mean()

In [ ]:
ss.head()

In [ ]:
submit = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': Y_pred})
submit.head()

In [ ]:
submit.shape

In [ ]:
submit.to_csv("final_submission.csv",index = False)